<a href="https://colab.research.google.com/github/Riyathapa305/IMDBdataset/blob/main/imdbdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
import zipfile
import io

zip_file_path = 'train.csv.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

extracted_files = os.listdir()
print("Extracted Files:", extracted_files)

csv_file_path = 'train.csv.zip'

train_df = pd.read_csv(csv_file_path)



Extracted Files: ['.config', 'train.csv.zip', '.ipynb_checkpoints', 'IMDB Dataset.csv', 'sample_data']


In [4]:
train_df.shape

(50000, 2)

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
le=LabelEncoder()
train_df['sentiment']=le.fit_transform(train_df['sentiment'])

In [7]:
train=train_df[:40000]

In [9]:
train.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1


In [8]:
sentiment=train['sentiment']

In [10]:
sentiment

0        1
1        1
2        1
3        0
4        1
        ..
39995    1
39996    1
39997    1
39998    0
39999    0
Name: sentiment, Length: 40000, dtype: int64

In [11]:
train=train['review']

In [31]:
sentiment

0      1
1      1
2      1
3      0
4      1
      ..
795    1
796    1
797    1
798    0
799    1
Name: sentiment, Length: 800, dtype: int64

In [12]:
train=train.str.lower()

In [13]:
import string
def remove_pun(text):
  st=string.punctuation
  return text.translate(str.maketrans('','',st))
train=train.apply(remove_pun)

In [14]:
pip install beautifulsoup4


In [15]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned_text = soup.get_text()
    return cleaned_text

In [16]:
train=train.apply(remove_html_tags)

In [17]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
def remove_stopwords(text):
  l=[]
  for i in text.split():
    if i in stopwords.words('english'):
      l.append('')
    else:
      l.append(i)
  return " ".join(l)
train=train.apply(remove_stopwords)

In [19]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lem=WordNetLemmatizer()
def words_lem(text):
  return " ".join([wordnet_lem.lemmatize(word) for word in text.split()])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
train=train.apply(words_lem)


In [21]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')


tokenized_reviews = [word_tokenize(review) for review in train]

model = Word2Vec(sentences=tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

word_embeddings = model.wv

embedding_for_movie = word_embeddings['movie']
print("Embedding for 'movie':", embedding_for_movie)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Embedding for 'movie': [-1.0160823  -1.3589013  -1.7405237   0.4408719   2.1484363  -1.6598125
 -0.74524343  1.8143988  -0.888104   -0.9182903   0.03638075  1.296982
  0.5000559   2.8504672   0.92268103 -1.3357245   2.229483    0.08688869
 -2.0776129  -1.9765911  -1.9391596  -1.1947465   1.8913842   2.5293941
 -0.6076873   0.15847427  0.18872389  0.34199023 -1.7610939   0.92416793
  1.8731053  -0.7842073   0.13962142 -0.00291653 -1.0603759   0.297745
 -1.2196528   0.80573165 -1.0174179   1.6697332  -0.61483103 -0.8791406
 -0.42170274  1.0510522   0.5178769   0.81038576 -0.07168169 -1.9017121
  0.7874096  -0.82605374  0.7277143  -2.6354818  -1.599435   -1.7193458
  1.3226798  -0.22268276 -0.09289342 -1.0032125  -2.1269374   0.9053268
 -0.72945154 -2.2271569   2.164933   -0.6899469  -1.0733937   1.3784652
 -0.08470984 -1.1397281  -0.9798938  -0.60870296 -0.10047787 -0.9021869
 -0.705046   -0.12148266 -1.1200786   1.5316737  -2.0144591  -0.55430186
  0.21392643  0.11151705 -1.1530328   1.

In [23]:
numeric_representations = [np.mean([word_embeddings[word] for word in tokens if word in word_embeddings], axis=0) for tokens in tokenized_reviews]


In [22]:
model.wv.most_similar('good')

[('decent', 0.7804273366928101),
 ('great', 0.7162587642669678),
 ('nice', 0.6902511119842529),
 ('bad', 0.6734520196914673),
 ('okay', 0.6520130038261414),
 ('fine', 0.6452736854553223),
 ('cool', 0.6432137489318848),
 ('ok', 0.6305968165397644),
 ('excellent', 0.6164004802703857),
 ('awesome', 0.6077532172203064)]

In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers



X_train, X_test, y_train, y_test = train_test_split(numeric_representations, sentiment, test_size=0.2, random_state=42)


model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train[0].shape[0],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(np.array(X_train), np.array(y_train), epochs=20, batch_size=32, validation_split=0.2)

y_pred = np.argmax(model.predict(np.array(X_test)), axis=1)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/20
800/800 [==============================] - 4s 4ms/step - loss: 0.4613 - accuracy: 0.7870 - val_loss: 0.3543 - val_accuracy: 0.8514
Epoch 2/20
800/800 [==============================] - 2s 3ms/step - loss: 0.3773 - accuracy: 0.8350 - val_loss: 0.3441 - val_accuracy: 0.8525
Epoch 3/20
800/800 [==============================] - 2s 3ms/step - loss: 0.3709 - accuracy: 0.8418 - val_loss: 0.3387 - val_accuracy: 0.8553
Epoch 4/20
800/800 [==============================] - 2s 3ms/step - loss: 0.3644 - accuracy: 0.8461 - val_loss: 0.3407 - val_accuracy: 0.8522
Epoch 5/20
800/800 [==============================] - 2s 3ms/step - loss: 0.3589 - accuracy: 0.8462 - val_loss: 0.3399 - val_accuracy: 0.8530
Epoch 6/20
800/800 [==============================] - 3s 4ms/step - loss: 0.3605 - accuracy: 0.8463 - val_loss: 0.3382 - val_accuracy: 0.8561
Epoch 7/20
800/800 [==============================] - 2s 3ms/step - loss: 0.3573 - accuracy: 0.8482 - val_loss: 0.3343 - val_accuracy: 0.8561
Epoch 

In [47]:
numeric_representations_str = [" ".join(map(str, representation)) for representation in numeric_representations]
